In [1]:
!pip install optuna lightgbm imbalanced-learn

  Using cached optuna-4.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached lightgbm-4.6.0-py3-none-win_amd64.whl.metadata (17 kB)
  Using cached imbalanced_learn-0.14.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached alembic-1.17.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached colorlog-6.10.1-py3-none-any.whl.metadata (11 kB)
  Using cached sqlalchemy-2.0.44-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached scipy-1.16.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached scikit_learn-1.7.2-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
  Using cached greenlet-3.2.4-cp313-cp313-win_amd64.whl.metadata (4.2 kB)
Using cached optuna-4.5.0-py3-none-any.whl (400 kB)
Using cached lightgbm-4.6.0-py3-none-win_amd64.whl (1.5 MB

In [5]:

import dagshub
dagshub.init(repo_owner='Vaibha3246', repo_name='influence_mirror', mlflow=True)

import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/Vaibha3246/influence_mirror.mlflow")

Initialized MLflow to track repo "Vaibha3246/influence_mirror"

Repository Vaibha3246/influence_mirror initialized!

In [6]:
# set or create an experiment
mlflow.set_experiment("exp_5 ml_algo_with_hp_tunning")


<Experiment: artifact_location='mlflow-artifacts:/adb82bab710d416190b0fea77cabca06', creation_time=1760159677050, experiment_id='3', last_update_time=1760159677050, lifecycle_stage='active', name='exp_5 ml_algo_with_hp_tunning', tags={}>

In [7]:
import pandas as pd
df=pd.read_csv('preprocessing.csv').dropna(subset=['text_clean'])

In [9]:
# -------------------------
# Core Libraries
# -------------------------
import numpy as np
import pandas as pd
import scipy.sparse as sp
import json
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# -------------------------
# Machine Learning & Preprocessing
# -------------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

# -------------------------
# Imbalanced Data Handling
# -------------------------
from imblearn.over_sampling import SMOTE

# -------------------------
# ML Models
# -------------------------
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# -------------------------
# Hyperparameter Optimization
# -------------------------
import optuna

# -------------------------
# Experiment Tracking
# -------------------------
import mlflow

# -------------------------
# Visualization Settings
# -------------------------
import warnings
warnings.filterwarnings('ignore')


In [16]:
# ML Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, early_stopping, log_evaluation


In [21]:
import json

In [25]:
# -------------------------
# Step 1: Clean target
# -------------------------
df['sentiment_numeric'] = df['sentiment_numeric'].map({-1: 2, 0: 0, 1: 1})
df = df.dropna(subset=['sentiment_numeric'])

# -------------------------
# Step 2: Select features
# -------------------------
numeric_cols = [col for col in df.columns if col not in [
    'video_id', 'category', 'text', 'text_clean', 'sentiment',
    'dominant_emotion', 'published_at', 'sentiment_numeric'
]]

X = df[numeric_cols]
y = df['sentiment_numeric']

# -------------------------
# Step 3: Train/test split
# -------------------------
X_train_raw, X_test_raw, y_train, y_test, train_idx, test_idx = train_test_split(
    X, y, df.index, test_size=0.2, random_state=42, stratify=y
)

# -------------------------
# Step 4: Scale numeric features
# -------------------------
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_raw)
X_test_num = scaler.transform(X_test_raw)

# -------------------------
# Step 5: TF-IDF for text
# -------------------------
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=10000)
X_train_text = tfidf.fit_transform(df.loc[train_idx, 'text_clean'])
X_test_text = tfidf.transform(df.loc[test_idx, 'text_clean'])

# Combine numeric + text
X_train = sp.hstack([X_train_text, sp.csr_matrix(X_train_num)])
X_test = sp.hstack([X_test_text, sp.csr_matrix(X_test_num)])

# -------------------------
# Step 6: Apply SMOTE
# -------------------------
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# -------------------------
# Step 7: Define Optuna objective for LightGBM
def objective(trial):
    params = {
        "objective": "multiclass",
        "num_class": len(np.unique(y)),
        "metric": "multi_logloss",
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 20, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 1.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 1.0),
        "verbose": -1,
        "n_jobs": -1,
        "seed": 42
    }

    model = LGBMClassifier(**params)
    model.fit(
        X_train_res, y_train_res,
        eval_set=[(X_test, y_test)],
        callbacks=[early_stopping(30), log_evaluation(0)]
    )

    preds = model.predict(X_test)
    f1 = f1_score(y_test, preds, average="macro")
    return f1
  

# -------------------------
# Step 8: Run Optuna Tuning
# -------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

# -------------------------
# Step 9: Train Best Model
# -------------------------
best_params = study.best_params
best_params.update({
    "objective": "multiclass",
    "num_class": len(np.unique(y)),
    "metric": "multi_logloss",
    "n_jobs": -1,
    "seed": 42
})

best_model = LGBMClassifier(**best_params)
best_model.fit(X_train_res, y_train_res)

y_pred = best_model.predict(X_test)

# -------------------------
# Step 10: Log Metrics + Model in MLflow
# -------------------------
with mlflow.start_run(run_name="LightGBM_TFIDF_SMOTE_Optuna_Best"):
    # Log algorithm name
    mlflow.log_param("algorithm", "LightGBM")

    # Log hyperparameters
    mlflow.log_params(best_params)
    mlflow.log_param("vectorizer_type", "TF-IDF")
    mlflow.log_param("ngram_range", (1, 3))
    mlflow.log_param("max_features", 10000)
    mlflow.log_param("imbalance_method", "SMOTE")

    # Log metrics
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_macro", f1_score(y_test, y_pred, average="macro"))

    # Log classification report
    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix: LightGBM Best Model")
    plt.savefig("confusion_matrix_lightgbm_best.png")
    mlflow.log_artifact("confusion_matrix_lightgbm_best.png")
    plt.close()

    # Save the model
    model_path = "lightgbm_best_model.pkl"
    joblib.dump(best_model, model_path)
    mlflow.log_artifact(model_path)

    # Log TF-IDF vocab
    vocab_json = {k: int(v) for k, v in tfidf.vocabulary_.items()}
    with open("tfidf_vocab.json", "w") as f:
     json.dump(vocab_json, f, indent=4)
    


    mlflow.log_artifact("tfidf_vocab.json")

print("🎯 MLflow run logged successfully for LightGBM ✅")


[I 2025-10-22 22:02:07,961] A new study created in memory with name: no-name-4bb6e778-bc76-444b-b346-8b48eb77135f


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[99]	valid_0's multi_logloss: 0.408772


[I 2025-10-22 22:02:15,777] Trial 0 finished with value: 0.7441304639444892 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 29, 'max_depth': 11, 'learning_rate': 0.13267889257882642, 'feature_fraction': 0.9858654121368846, 'bagging_fraction': 0.9066444869503921, 'bagging_freq': 4, 'min_data_in_leaf': 25, 'lambda_l1': 0.37724252048500884, 'lambda_l2': 0.767700367187117}. Best is trial 0 with value: 0.7441304639444892.


Training until validation scores don't improve for 30 rounds


[I 2025-10-22 22:02:24,114] Trial 1 finished with value: 0.7290159635320415 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 48, 'max_depth': 15, 'learning_rate': 0.015169066444619591, 'feature_fraction': 0.953796524207746, 'bagging_fraction': 0.6144724273059273, 'bagging_freq': 3, 'min_data_in_leaf': 33, 'lambda_l1': 0.34649282713660245, 'lambda_l2': 0.3447942000484161}. Best is trial 0 with value: 0.7441304639444892.


...

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.417154


[I 2025-10-22 22:05:07,606] Trial 23 finished with value: 0.7396170983874892 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 58, 'max_depth': 7, 'learning_rate': 0.08102572891363255, 'feature_fraction': 0.6992556589661127, 'bagging_fraction': 0.6531453037339374, 'bagging_freq': 4, 'min_data_in_leaf': 17, 'lambda_l1': 0.575903144143839, 'lambda_l2': 0.6979589915774849}. Best is trial 14 with value: 0.748546629780319.


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.407439


[I 2025-10-22 22:05:18,000] Trial 24 finished with value: 0.7477712829079972 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 171, 'max_depth': 10, 'learning_rate': 0.1684189735962493, 'feature_fraction': 0.6228814912032165, 'bagging_fraction': 0.8176962704348362, 'bagging_freq': 3, 'min_data_in_leaf': 20, 'lambda_l1': 0.41874883836163174, 'lambda_l2': 0.8936359971372222}. Best is trial 14 with value: 0.748546629780319.


🏃 View run LightGBM_TFIDF_SMOTE_Optuna_Best at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/3/runs/6dfa4cb7beb849c9b7c76f010ff730e5
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/3
🎯 MLflow run logged successfully for LightGBM ✅
